In [1]:
import time

# pour un ordi avec GPU :

import GPUtil
import torch
print(torch.cuda.is_available())
!set CUDA_PATH="/usr/local/cuda-12"

import pandas as pd
import spacy

GPUtil.showUtilization()

True
| ID | GPU | MEM |
------------------
|  0 | 38% |  2% |


In [2]:
# vérifier l'état des données

def check_data(df_to_check):

    if df_to_check.isnull().values.any() :
        print("y a t il des nan? ->",df_to_check.isnull().values.any())
        print("combien y a t il de nan? ->",df_to_check.isnull().values.sum())
        print("où sont les null? ->\n",df_to_check.isnull().sum())
        print(df_to_check[df_to_check.isnull().T.any()])
    else :
        print("il n'y a pas de NaN")

    print("quels types pour chaque colonne?\n")    
    for column in df_to_check.columns:
        print(pd.api.types.infer_dtype(df_to_check[column]))
    print("y a-t-il des types mixés dans les données?\n")
    print(df_to_check[column][df_to_check[column].apply(lambda x: isinstance(x, type))])
    print(df_to_check._is_mixed_type)
    print(df_to_check.dtypes.nunique()>1)


In [3]:
df = pd.read_csv("df_CC_lemmas_postag.csv", sep = ",", encoding = "utf-8", dtype= str)

In [4]:
#  j'enlève les colonnes que je n'utiliserai pas avec BERTopic, pour réduire le poids du fichier 
df = df.drop(["Contribution","lemmas", "postags", "lemmas_only_VERB_ADJ_ADV_NOUN", "postags_only_VERB_ADJ_ADV_NOUN", "lemmas_no_stopwords", "clean_no_stopwords", "VERB_ADJ_ADV_NOUN_no_stopwords"], axis=1)
# df = df.drop(["Contribution"], axis=1)


In [5]:
#  certaines lignes n'ont que des NaN pour les stopwords, c'est parce que la contribution était seulement composée de illisibles et de stopwords
df = df.dropna()

# on élimine les rang qui ont des nan, 
# en conservant l'index originel

In [6]:
# si on souhaite print des contributions dans toute leur longueur (ne pas appliquer sur tout le corpus)

def print_full(df):
    pd.set_option('display.max_colwidth', 3000)
    # pd.set_option('display.max_rows', None)
    # pd.set_option('display.max_columns', None)
    # pd.set_option('display.width', 2000)
    # pd.set_option('display.float_format', '{:20,.2f}'.format)
    # pd.set_option('display.max_colwidth', None)
    print(df)
    # pd.reset_option('display.max_rows')
    # pd.reset_option('display.max_columns')
    # pd.reset_option('display.width')
    # pd.reset_option('display.float_format')
    # pd.reset_option('display.max_colwidth')
    pd.reset_option('display.max_colwidth')

# autres options :


In [7]:
# rename the columns without spaces (header names, not my best idea)

df = df.rename(columns={"Date de réception" : "Date_de_reception", "Code postal" : "Code_postal", "Code INSEE" : "Code_INSEE",
           "Numéro d'ordre arbitraire" : "Numero_d_ordre_arbitraire", "Type Graphie TT" : "Type_Graphie_TT" , "Numéro de page" : "Numero_de_page",
           "Numéro séquentiel" : "Numero_sequentiel"})

In [8]:
nlp = spacy.load("fr_core_news_md", disable = ["ner"])



In [9]:
# config = en commentaire, l'ensemble de règles sentencize(), sinon le modèle sents

def sentencize(document, subset, column_name):
    if subset :
        document = document[0:subset]

    t0 = time.time()
    all_sentences_df = pd.DataFrame()

    for doc, infos_doc in zip(nlp.pipe(document[column_name], batch_size=128), document.itertuples()):
        sentence_list = [sent.text.strip() for sent in doc.sents]

        sentence_df = pd.DataFrame({"phrases" : sentence_list})

        metadata = pd.DataFrame([infos_doc]*len(sentence_list))

        sentence_df = pd.concat([sentence_df,metadata], axis=1)

        all_sentences_df = pd.concat([all_sentences_df,sentence_df])

        # deleting the lists for clearing, to force python garbage collector to *actually* collect

        del sentence_df
        del sentence_list
        del metadata

            
    t3 = time.time()
    print("Total time: {}".format(t3-t0))
    #  ? all_sentences_df = all_sentences_df.drop(labels = "Contribution", axis=1)
    return(all_sentences_df)


In [10]:
# this function ASKS 10h to be run on a large dataset
# DO NOT FORGET TO CHOOSE A SUBSET size IF TESTING

sentences = sentencize(df[["Catégorie","Date_de_reception","Code_postal","Code_INSEE","Numero_d_ordre_arbitraire",
                          "Type_Graphie_TT","Numero_de_page","Numero_sequentiel","clean"]],subset = None, column_name = "clean")

Total time: 13373.065524339676


In [11]:
# adding a column to the df
# renumber the index column (not accurate bc origin from each concatenation)

sentence_df = sentences.reset_index()

# drop the secoond index column

sentence_df = sentence_df.drop(labels = "index", axis=1)

In [13]:
# size of sentence_df at the end : (2401714, 11)

# turn to pickle
sentence_df.to_pickle("pickle_sentences_df.pkl")

In [14]:
# first version was saved as csv
# default way should be to turn to saved to pickle since the beginnning

# here I dowloaded csv format for sentences then turned it to pickle for using after

# df = pd.read_csv("sentences_df.csv", sep = ",", encoding = "utf-8", dtype= str)
# df.to_pickle("pickle_sentences_sentences.pkl")
